In [9]:
import tensorflow as tf
import numpy as np
import pickle

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/Users/siva/Documents/thesis/datasets/MNIST", one_hot=True)

Extracting /Users/siva/Documents/thesis/datasets/MNIST/train-images-idx3-ubyte.gz
Extracting /Users/siva/Documents/thesis/datasets/MNIST/train-labels-idx1-ubyte.gz
Extracting /Users/siva/Documents/thesis/datasets/MNIST/t10k-images-idx3-ubyte.gz
Extracting /Users/siva/Documents/thesis/datasets/MNIST/t10k-labels-idx1-ubyte.gz


In [11]:
def weights_(shape):
    return tf.Variable(tf.random_normal(shape,stddev = 0.01))
    #return tf.Variable(tf.constant(2.0,shape = shape))
def biases_(shape):
    return tf.Variable(tf.random_normal(shape,stddev = 0.01))
def conv2d_(inp_,w_):
    return tf.nn.conv2d(inp_,w_,strides = [1,1,1,1],padding = "SAME")
def unpickle(file_name):
    with open(file_name,"rb") as fo:
        dict_ = pickle.load(fo,encoding = "bytes")
        temp = dict_[b'data'][:]
        dict_[b'data'] = np.reshape(temp,(-1,32,32,3))
    return dict_

In [12]:
def model_final(FILTER_SIZE,MAX_FILTERS,RATE_OF_NEURONS,LEARNING_RATE,INPUT_DIM = [None,28,28,1],OUTPUT_DIM = [None,10]):
    """
    FILTER_SIZE: size of the filters used in convolution
    MAX_FILTERS: maximum number of filters initialized
    RATE_OF_NEURONS: rate of increase of neurons
    LEARNING_RATE: learning rate of the optimizer
    INPUT_DIM: input dimension of placeholder x
    OUTPUT_DIM: output dimension of placeholder y
    """
    x = tf.placeholder(tf.float32,INPUT_DIM)
    y = tf.placeholder(tf.float32,OUTPUT_DIM)
    
    no_of_filters = tf.placeholder(tf.int32)
    mask = tf.placeholder(tf.int32)
    
    conv_w_original = weights_([FILTER_SIZE,FILTER_SIZE,3,MAX_FILTERS])
    conv_b_original = biases_([MAX_FILTERS])
    
    #conv_w = tf.slice(conv_w_original,(0,0,0,0),(FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],no_of_filters))
    #conv_b = tf.slice(conv_b_original,(0),(no_of_filters))
    conv_w = conv_w_original[0:FILTER_SIZE,0:FILTER_SIZE,0:tf.shape(x)[-1],0:no_of_filters]
    conv_b = conv_b_original[0:no_of_filters]
    
    conv_w_mask = tf.concat([tf.ones([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],no_of_filters-RATE_OF_NEURONS]),\
                             tf.zeros([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],RATE_OF_NEURONS])],3)
    conv_b_mask = tf.concat([tf.ones([no_of_filters-RATE_OF_NEURONS]),tf.zeros([RATE_OF_NEURONS])],0)
    conv_w_mask_not = tf.concat([tf.zeros([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],no_of_filters-RATE_OF_NEURONS]),\
                                 tf.ones([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],RATE_OF_NEURONS])],3)
    conv_b_mask_not = tf.concat([tf.zeros([no_of_filters-RATE_OF_NEURONS]),tf.ones([RATE_OF_NEURONS])],0)
    
    #depending on the mask value gradient update is done
    w = tf.cond(tf.equal(mask,1),lambda:tf.stop_gradient(conv_w * conv_w_mask) + (conv_w * conv_w_mask_not),\
                lambda:conv_w)
    b = tf.cond(tf.equal(mask,1),lambda:tf.stop_gradient(conv_b * conv_b_mask) + (conv_b * conv_b_mask_not),\
                lambda:conv_b)
        
    conv_out = tf.nn.relu(tf.nn.bias_add(conv2d_(x,w),b))
    
    full_w_original = weights_([INPUT_DIM[1]*INPUT_DIM[2]*MAX_FILTERS,10])
    #full_w = tf.slice(full_w_original,(0,0),(INPUT_DIM[1]*INPUT_DIM[2]*no_of_filters,10))
    full_w = full_w_original[0:(INPUT_DIM[1]*INPUT_DIM[2]*no_of_filters),0:10]
    full_b = biases_([10])
    
    out = tf.matmul(tf.reshape(conv_out,(-1,INPUT_DIM[1]*INPUT_DIM[2]*no_of_filters)),full_w) + full_b
    #softmax_out = tf.nn.softmax(out)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out,labels = y))
    prediction = tf.equal(tf.argmax(out,axis = 1),tf.argmax(y, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
    
    #summary writing
    tf.summary.scalar("loss",loss)
    tf.summary.scalar("acc",accuracy)
    tensor_b = tf.summary.merge_all()
    
    return {"inputs":[x,y,no_of_filters,mask],"weight":conv_w,"optimizer":optimizer,"loss":loss,"output":conv_out,"accuracy":accuracy,"tensor_b":tensor_b}

In [13]:
def train_cifar(learning_rate):
    DATA_FOLDER = "/Users/siva/Documents/thesis/datasets/cifar_10/"
    TENSORBOARD_FOLDER = "/Users/siva/Documents/thesis/tensorboard"
    EPOCHS = 10
    BATCH_SIZE = 100
    #loading the data
    for i in range(5):
        if i == 0:
            data = unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'data']
            labels = unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'labels']
            labels = np.array(list(map(lambda x:np.eye(10)[x],labels)))
        else:
            data = np.append(data,unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'data'],axis = 0)
            labels_ = unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'labels']
            labels = np.append(labels,np.array(list(map(lambda x:np.eye(10)[x],labels_))),axis = 0)
    test_data = unpickle(DATA_FOLDER+"test_batch")[b'data']
    test_labels = unpickle(DATA_FOLDER+"test_batch")[b'labels']
    test_labels = np.array(list(map(lambda x:np.eye(10)[x],test_labels)))
    print(test_data.shape)
    print(test_labels.shape)
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        cifar_model = model_final(3,300,1,learning_rate,[None,32,32,3])
        filters = 256
        mask = 0
        sess.run(tf.global_variables_initializer())
        train_writer = tf.summary.FileWriter(TENSORBOARD_FOLDER+"/train",sess.graph)
        test_writer = tf.summary.FileWriter(TENSORBOARD_FOLDER+"/test",sess.graph)
        train_count = 0
        test_count = 0
        for e in range(EPOCHS):
            train_loss_list = []
            train_acc_list = []
            test_loss_list = []
            test_acc_list = []
            for b in range(int(data.shape[0]/BATCH_SIZE)):
                train_data = data[b:b+BATCH_SIZE,:]
                train_labels = labels[b:b+BATCH_SIZE,:]
                to_compute = [cifar_model["optimizer"],cifar_model["loss"],cifar_model["accuracy"],cifar_model["tensor_b"]]
                feed_dict_ = dict(zip(cifar_model["inputs"],[train_data,train_labels,filters,mask]))
                _,train_loss,train_acc,train_summ = sess.run(to_compute,feed_dict = feed_dict_)
                train_loss_list.append(train_loss)
                train_acc_list.append(train_acc)
                #train_writer.add_summary(train_summ,train_count)
                train_count += 1
            print("train:",np.mean(train_loss_list),"|",np.mean(train_acc_list))
            for t in range(int(test_data.shape[0]/BATCH_SIZE)):
                test_data_ = test_data[t:t+BATCH_SIZE,:]
                test_labels_ = test_labels[t:t+BATCH_SIZE,:]
                to_compute_t = [cifar_model["loss"],cifar_model["accuracy"],cifar_model["tensor_b"]]
                feed_dict_t = dict(zip(cifar_model["inputs"],[test_data_,test_labels_,filters,mask]))
                test_loss,test_acc,test_summ = sess.run(to_compute_t,feed_dict = feed_dict_t)
                test_loss_list.append(test_loss)
                test_acc_list.append(test_acc)
                #test_writer.add_summary(test_summ,test_count)
                test_count += 1
            print("test:",np.mean(test_loss_list),"|",np.mean(test_acc_list))

In [17]:
def train_mnist(learning_rate):
    DATA_FOLDER = "/Users/siva/Documents/thesis/datasets/"
    TENSORBOARD_FOLDER = "/Users/siva/Documents/thesis/tensorboard"
    EPOCHS = 100
    BATCH_SIZE = 100
    #loading the data
#     for i in range(5):
#         if i == 0:
#             data = unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'data']
#             labels = unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'labels']
#             labels = np.array(list(map(lambda x:np.eye(10)[x],labels)))
#         else:
#             data = np.append(data,unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'data'],axis = 0)
#             labels_ = unpickle(DATA_FOLDER+"data_batch_{0}".format(i+1))[b'labels']
#             labels = np.append(labels,np.array(list(map(lambda x:np.eye(10)[x],labels_))),axis = 0)
#     test_data = unpickle(DATA_FOLDER+"test_batch")[b'data']
#     test_labels = unpickle(DATA_FOLDER+"test_batch")[b'labels']
#     test_labels = np.array(list(map(lambda x:np.eye(10)[x],test_labels)))
    data = np.reshape(mnist.train.images,(-1,28,28,1))
    labels = mnist.train.labels
    test_data = np.reshape(mnist.test.images,(-1,28,28,1))
    test_labels = mnist.test.labels
    print(test_data.shape)
    print(test_labels.shape)
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        mnist_model = model_final(3,300,1,learning_rate,[None,28,28,1])
        filters = 256
        mask = 0
        sess.run(tf.global_variables_initializer())
        train_writer = tf.summary.FileWriter(TENSORBOARD_FOLDER+"/train",sess.graph)
        test_writer = tf.summary.FileWriter(TENSORBOARD_FOLDER+"/test",sess.graph)
        train_count = 0
        test_count = 0
        for e in range(EPOCHS):
            train_loss_list = []
            train_acc_list = []
            test_loss_list = []
            test_acc_list = []
            for b in range(int(data.shape[0]/BATCH_SIZE)):
                train_data = data[b:b+BATCH_SIZE,:]
                train_labels = labels[b:b+BATCH_SIZE,:]
                to_compute = [mnist_model["optimizer"],mnist_model["loss"],mnist_model["accuracy"],mnist_model["tensor_b"]]
                feed_dict_ = dict(zip(mnist_model["inputs"],[train_data,train_labels,filters,mask]))
                _,train_loss,train_acc,train_summ = sess.run(to_compute,feed_dict = feed_dict_)
                train_loss_list.append(train_loss)
                train_acc_list.append(train_acc)
                #train_writer.add_summary(train_summ,train_count)
                train_count += 1
            print("train:",np.mean(train_loss_list),"|",np.mean(train_acc_list))
            for t in range(int(test_data.shape[0]/BATCH_SIZE)):
                test_data_ = test_data[t:t+BATCH_SIZE,:]
                test_labels_ = test_labels[t:t+BATCH_SIZE,:]
                to_compute_t = [mnist_model["loss"],mnist_model["accuracy"],mnist_model["tensor_b"]]
                feed_dict_t = dict(zip(mnist_model["inputs"],[test_data_,test_labels_,filters,mask]))
                test_loss,test_acc,test_summ = sess.run(to_compute_t,feed_dict = feed_dict_t)
                test_loss_list.append(test_loss)
                test_acc_list.append(test_acc)
                #test_writer.add_summary(test_summ,test_count)
                test_count += 1
            print("test:",np.mean(test_loss_list),"|",np.mean(test_acc_list))

In [18]:
lr = np.linspace(-6,6,13)
lr = 10**lr
for lr_ in lr:
    print("*******learning rate: {0}".format(str(lr_)))
    train_mnist(lr_)

*******learning rate: 1e-06
(10000, 28, 28, 1)
(10000, 10)
train: 2.24821 | 0.266436
test: 2.23831 | 0.3545
train: 2.13466 | 0.476709
test: 2.14539 | 0.5147
train: 1.9807 | 0.643
test: 2.01241 | 0.5788


KeyboardInterrupt: 

In [3]:
def train():
    #data = np.random.normal(0,1,[2,3,3,2])
    data = np.reshape(np.arange(36),(2,3,3,2))
    labels = np.ones((2,10))
    tf.reset_default_graph()
    with tf.Session() as sess:
        my_model = model_dummy(2,10,1,1)
        sess.run(tf.global_variables_initializer())
        w_prev = sess.run(my_model["weight"],feed_dict = \
                          {my_model["inputs"][0]:data,my_model["inputs"][1]:labels,\
                           my_model["inputs"][2]:8,my_model["inputs"][3]:1})
        print(w_prev)
        for i in range(10):
            to_compute = [my_model["output"],my_model["weight"],my_model["optimizer"]]
            feed_dict_ = {my_model["inputs"][0]:data,my_model["inputs"][1]:labels,my_model["inputs"][2]:8,my_model["inputs"][3]:1}
            out_,w_,_ = sess.run(to_compute,feed_dict = feed_dict_)
            #print(out_[0])
            #print(out_)
            print(w_)

In [4]:
def model_dummy(FILTER_SIZE,MAX_FILTERS,RATE_OF_NEURONS,LEARNING_RATE,INPUT_DIM = [None,3,3,2],OUTPUT_DIM = [None,10]):
    """
    FILTER_SIZE: size of the filters used in convolution
    MAX_FILTERS: maximum number of filters initialized
    RATE_OF_NEURONS: rate of increase of neurons
    LEARNING_RATE: learning rate of the optimizer
    INPUT_DIM: input dimension of placeholder x
    OUTPUT_DIM: output dimension of placeholder y
    """
    x = tf.placeholder(tf.float32,INPUT_DIM)
    y = tf.placeholder(tf.float32,OUTPUT_DIM)
    
    no_of_filters = tf.placeholder(tf.int32)
    mask = tf.placeholder(tf.int32)
    
    conv_w_original = weights_([FILTER_SIZE,FILTER_SIZE,3,MAX_FILTERS])
    conv_b_original = biases_([MAX_FILTERS])
    
    #conv_w = tf.slice(conv_w_original,(0,0,0,0),(FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],no_of_filters))
    conv_w = conv_w_original[0:FILTER_SIZE,0:FILTER_SIZE,0:tf.shape(x)[-1],0:no_of_filters]
    conv_b = conv_b_original[0:no_of_filters]
    
    conv_w_mask = tf.concat([tf.ones([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],no_of_filters-RATE_OF_NEURONS]),\
                             tf.zeros([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],RATE_OF_NEURONS])],3)
    conv_b_mask = tf.concat([tf.ones([no_of_filters-RATE_OF_NEURONS]),tf.zeros([RATE_OF_NEURONS])],0)
    conv_w_mask_not = tf.concat([tf.zeros([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],no_of_filters-RATE_OF_NEURONS]),\
                                 tf.ones([FILTER_SIZE,FILTER_SIZE,tf.shape(x)[-1],RATE_OF_NEURONS])],3)
    conv_b_mask_not = tf.concat([tf.zeros([no_of_filters-RATE_OF_NEURONS]),tf.ones([RATE_OF_NEURONS])],0)
    
    #depending on the mask value gradient update is done
    w = tf.cond(tf.equal(mask,1),lambda:tf.stop_gradient(conv_w * conv_w_mask) + (conv_w * conv_w_mask_not),\
                lambda:conv_w)
    b = tf.cond(tf.equal(mask,1),lambda:tf.stop_gradient(conv_b * conv_b_mask) + (conv_b * conv_b_mask_not),\
                lambda:conv_b)
        
    conv_out = tf.nn.relu(tf.nn.bias_add(conv2d_(x,w),b))
    
    full_w_original = weights_([INPUT_DIM[1]*INPUT_DIM[2]*MAX_FILTERS,10])
    full_w = tf.slice(full_w_original,(0,0),(INPUT_DIM[1]*INPUT_DIM[2]*no_of_filters,10))
    
    out = tf.matmul(tf.reshape(conv_out,(-1,INPUT_DIM[1]*INPUT_DIM[2]*no_of_filters)),full_w)
    #softmax_out = tf.nn.softmax(out)
    
    loss = out - y
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out,labels = y))
    prediction = tf.equal(tf.argmax(out,axis = 1),tf.argmax(y, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
    
    #summary writing
    tf.summary.scalar("loss",loss)
    tf.summary.scalar("acc",accuracy)
    tensor_b = tf.summary.merge_all()
    
    return {"inputs":[x,y,no_of_filters,mask],"weight":b,"optimizer":optimizer,"loss":loss,"output":conv_out,"accuracy":accuracy,"tensor_b":tensor_b}

In [5]:
train()

[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.01502581 -0.00239594 -0.01728372 -0.02503985 -0.00987763 -0.00050823
  0.02000872 -0.0014663 ]
[ 0.015025